In [1]:
from __future__ import print_function

from keras import utils
from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.optimizers import SGD, Adam

import numpy as np

from PIL import Image, ImageOps

import argparse
import math
import os
import os.path
import glob

K.set_image_dim_ordering('th') # ensure our dimension notation matches

Using TensorFlow backend.


In [2]:
G_WGTS_FILE = 'goodgenerator.h5'
D_WGTS_FILE = 'gooddiscriminator.h5'

ENTROPY = 'binary_crossentropy'
SGD_OPT = 'SGD'

EPOCHS = 400
BATCH_SIZE = 10

In [3]:
!mkdir logo-generated-images

mkdir: cannot create directory ‘logo-generated-images’: File exists


## Loading Data

In [4]:
class DataLoader:
    
    def get_train_image_set(self):
        '''
        Return formatted train image set from loaded dataset.
        '''
        X_train = self.load_train_image_data()
        X_train = (X_train.astype(np.float32)-127.5)/127.5
        X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:])
        print('X_train.shape: {}'.format(X_train.shape))
        return X_train
    
    
    def load_train_image_data(self, pixels=128, verbose=False):
        '''
        Load image dataset from directory.
        '''
        print("Loading data...")
        
        X_train = []
        paths = glob.glob(os.path.normpath(os.getcwd()+'/logos/*.jpg'))
        
        for path in paths:
            if verbose: print(path)
            
            im = Image.open(path)
            im = ImageOps.fit(im, (pixels, pixels), Image.ANTIALIAS)
            im = ImageOps.grayscale(im)
            im = np.asarray(im)
            
            X_train.append(im)
            
        print("Finished loading data")
        
        return np.array(X_train)

In [5]:
dataLoader = DataLoader()

X_train = dataLoader.get_train_image_set()

Loading data...
Finished loading data
X_train.shape: (38, 1, 128, 128)


## Creating Models

In [6]:
class ModelCreator:
    
    def generator_model(self):
        '''
        Build generator model.
        '''
        model = Sequential()
        model.add(Dense(input_dim=100, units=1024))
        model.add(Activation('tanh'))
        model.add(Dense(128*8*8))
        model.add(BatchNormalization())
        model.add(Activation('tanh'))
        model.add(Reshape((128, 8, 8), input_shape=(128*8*8,)))
        model.add(UpSampling2D(size=(4, 4)))
        model.add(Convolution2D(64, (5, 5), padding='same'))
        model.add(Activation('tanh'))
        model.add(UpSampling2D(size=(4, 4)))
        model.add(Convolution2D(1, (5, 5), padding='same'))
        model.add(Activation('tanh'))
        return model
    
    
    def discriminator_model(self):
        '''
        Build discriminator model.
        '''
        model = Sequential()
        model.add(Convolution2D(64, (5, 5), padding='same', input_shape=(1, 128, 128)))
        model.add(Activation('tanh'))
        model.add(AveragePooling2D(pool_size=(4, 4)))
        model.add(Convolution2D(128, (5, 5)))
        model.add(Activation('tanh'))
        model.add(AveragePooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(256))
        model.add(Activation('tanh'))
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        return model
    
    
    def generator_containing_discriminator(self, generator, discriminator):
        '''
        Build generator model containing discriminator.
        '''
        model = Sequential()
        model.add(generator)
        discriminator.trainable = False
        model.add(discriminator)
        return model

In [7]:
class ModelCompiler:
    
    def compile_models(self, load_weights=False):
        '''
        Build and compile models.
        
        :param load_weights: Indicates whether or not to load weights from files.
        '''
        
        modelCreator = ModelCreator()
        
        # Create models
        generator = modelCreator.generator_model()
        discriminator = modelCreator.discriminator_model()
        
        # Load weights
        if load_weights:
            self.load_weights(generator, discriminator)
  
        # Create generator containing discriminator
        discriminator_on_generator = \
            modelCreator.generator_containing_discriminator(generator, discriminator)

        # Add losses and optimizers
        g_opt = SGD(lr=0.0005, momentum=0.9, nesterov=True)
        d_opt = SGD(lr=0.0005, momentum=0.9, nesterov=True)
        
        generator.compile(loss=ENTROPY, optimizer=SGD_OPT)
        discriminator_on_generator.compile(loss=ENTROPY, optimizer=g_opt)
        discriminator.trainable = True
        discriminator.compile(loss=ENTROPY, optimizer=d_opt)
        
        return (generator, discriminator, discriminator_on_generator)
    
        
    def load_weights(self, generator, discriminator):
        '''
        Load weights from files.
        '''
        generator.load_weights(G_WGTS_FILE, True)
        discriminator.load_weights(D_WGTS_FILE, True)
        
        
    def print_models(self, generator, discriminator, discriminator_on_generator):
        print('\ngenerator:')
        generator.summary()
        print('\n\ndiscriminator:')
        discriminator.summary()
        print('\n\ndiscriminator_on_generator:')
        discriminator_on_generator.summary()
        print('')

In [8]:
modelCompiler = ModelCompiler()

models = modelCompiler.compile_models()

modelCompiler.print_models(*models)


generator:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              103424    
_________________________________________________________________
activation_1 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8192)              8396800   
_________________________________________________________________
batch_normalization_1 (Batch (None, 8192)              32768     
_________________________________________________________________
activation_2 (Activation)    (None, 8192)              0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 128, 8, 8)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 128, 32, 32)       0        

In [19]:
class ModelTrainer:
    
    def train_models(self, models, X_train, load_weights=False):
        '''
        Train models.
        
        :param models: Models to train
        :param X_train: Training image dataset
        :param load_weights: Whether or not to create models with weights loaded from files
        '''
        
        # Load models with saved weights
        if load_weights:
            models = ModelCompiler().compile_models(True)
        
        generator = models[0]
        discriminator = models[1]
        discriminator_on_generator = models[2]
        
        imageProcessor = ImageProcessor()
        
        # Initialize noise
        noise = np.zeros((BATCH_SIZE, 100))
        
        for epoch in range(EPOCHS):
            print('\nEpoch: {}'.format(epoch+1))
            
            n_batches = int(X_train.shape[0]/BATCH_SIZE)
            
            for idx in range(n_batches):
                # Randomize noise
                Util().randomize_noise(noise, BATCH_SIZE)
                    
                image_batch = X_train[idx*BATCH_SIZE:(idx+1)*BATCH_SIZE]
                generated_images = generator.predict(noise, verbose=0)
                
                # Save image
                if idx % 20 == 0 and epoch % 10 == 0:
                    imageProcessor.save_image(epoch, idx, generated_images)
                
                # Compute and log discriminator loss
                self.compute_d_loss(idx, discriminator, image_batch, generated_images)        
               
                # Randomize noise
                Util().randomize_noise(noise, BATCH_SIZE)
            
                # Compute and log generator loss
                self.compute_g_loss(idx, discriminator, discriminator_on_generator, noise)
            
                if epoch % 10 == 9:
                    self.save_weights(generator, discriminator)

        print('\nTraining complete\n')
        

    def compute_d_loss(self, idx, discriminator, image_batch, generated_images):
        '''
        Compute discriminator loss.
        '''
        X = np.concatenate((image_batch, generated_images))
        y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
        d_loss = discriminator.train_on_batch(X, y)
        print("batch %d d_loss : %f" % (idx+1, d_loss))

        
    def compute_g_loss(self, idx, discriminator, discriminator_on_generator, noise):
        '''
        Compute generator loss.
        '''
        discriminator.trainable = False
        g_loss = discriminator_on_generator.train_on_batch(noise, [1]*BATCH_SIZE)
        discriminator.trainable = True
        print("batch %d g_loss : %f" % (idx+1, g_loss))
    
    
    def save_weights(self, generator, discriminator):
        '''
        Save trained weights to files.
        '''
        generator.save_weights(G_WGTS_FILE)
        discriminator.save_weights(D_WGTS_FILE)

In [20]:
class Util:
    
    def randomize_noise(self, noise, batch_size):
        '''
        Randomize noise matrix.
        '''
        for i in range(batch_size):
            noise[i, :] = np.random.uniform(-1, 1, 100)

In [11]:
class ImageProcessor:
    
    def save_image(self, epoch, idx, images):
        '''
        Save generated images.
        '''
        image = self.combine_images(images)
        image = image * 127.5 + 127.5

        path_str = os.getcwd() + '/logo-generated-images/'\
                               + str(epoch+1)+'_'+str(idx)+'.png'
        dst_path = os.path.normpath(path_str)
        
        Image.fromarray(image.astype(np.uint8)).save(dst_path)

        
    def combine_images(self, images):
        '''
        Combine generated images.
        '''
        num = images.shape[0]
        width = int(math.sqrt(num))
        height = int(math.ceil(float(num)/width))
        shape = images.shape[2:]
        
        image = np.zeros((height*shape[0], width*shape[1]),
                          dtype=images.dtype)
        
        for idx, img in enumerate(images):
            i = int(idx/width)
            j = idx % width
            
            image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = img[0, :, :]
        
        return image
    
    
    def clean(self, image):
        '''
        Clean image.
        '''
        for i in range(1, image.shape[0]-1):
            for j in range(1, image.shape[1]-1):
                if image[i][j] \
                        + image[i+1][j] + image[i][j+1] \
                        + image[i-1][j] + image[i][j-1] > 127 * 5:
                    image[i][j] = 255

In [12]:
modelTrainer = ModelTrainer()

modelTrainer.train_models(models, X_train, False)


Epoch: 1
batch 1 d_loss : 0.698320
batch 1 g_loss : 0.690590
batch 2 d_loss : 0.666958
batch 2 g_loss : 0.693864
batch 3 d_loss : 0.618211
batch 3 g_loss : 0.686351

Epoch: 2
batch 1 d_loss : 0.576769
batch 1 g_loss : 0.692502
batch 2 d_loss : 0.530379
batch 2 g_loss : 0.692041
batch 3 d_loss : 0.492321
batch 3 g_loss : 0.697117

Epoch: 3
batch 1 d_loss : 0.459148
batch 1 g_loss : 0.715395
batch 2 d_loss : 0.425645
batch 2 g_loss : 0.695003
batch 3 d_loss : 0.412032
batch 3 g_loss : 0.711067

Epoch: 4
batch 1 d_loss : 0.403215
batch 1 g_loss : 0.694587
batch 2 d_loss : 0.391371
batch 2 g_loss : 0.707660
batch 3 d_loss : 0.400309
batch 3 g_loss : 0.675526

Epoch: 5
batch 1 d_loss : 0.413974
batch 1 g_loss : 0.691071
batch 2 d_loss : 0.390900
batch 2 g_loss : 0.681379
batch 3 d_loss : 0.395451
batch 3 g_loss : 0.657195

Epoch: 6
batch 1 d_loss : 0.419178
batch 1 g_loss : 0.690335
batch 2 d_loss : 0.426305
batch 2 g_loss : 0.660186
batch 3 d_loss : 0.440412
batch 3 g_loss : 0.663432

Epo

batch 2 d_loss : 0.280430
batch 2 g_loss : 1.067245
batch 3 d_loss : 0.179315
batch 3 g_loss : 1.250688

Epoch: 51
batch 1 d_loss : 0.278445
batch 1 g_loss : 1.364216
batch 2 d_loss : 0.332016
batch 2 g_loss : 1.187241
batch 3 d_loss : 0.159855
batch 3 g_loss : 1.151769

Epoch: 52
batch 1 d_loss : 0.271874
batch 1 g_loss : 0.803803
batch 2 d_loss : 0.376958
batch 2 g_loss : 0.751530
batch 3 d_loss : 0.226965
batch 3 g_loss : 1.003902

Epoch: 53
batch 1 d_loss : 0.341675
batch 1 g_loss : 1.175690
batch 2 d_loss : 0.439579
batch 2 g_loss : 1.436688
batch 3 d_loss : 0.531231
batch 3 g_loss : 0.996105

Epoch: 54
batch 1 d_loss : 0.347825
batch 1 g_loss : 1.253635
batch 2 d_loss : 0.425741
batch 2 g_loss : 0.776099
batch 3 d_loss : 0.263948
batch 3 g_loss : 0.949561

Epoch: 55
batch 1 d_loss : 0.327786
batch 1 g_loss : 1.097824
batch 2 d_loss : 0.612590
batch 2 g_loss : 0.879674
batch 3 d_loss : 0.322797
batch 3 g_loss : 1.037448

Epoch: 56
batch 1 d_loss : 0.329610
batch 1 g_loss : 1.08409

batch 3 d_loss : 0.451809
batch 3 g_loss : 1.515083

Epoch: 100
batch 1 d_loss : 0.367593
batch 1 g_loss : 1.343374
batch 2 d_loss : 0.398831
batch 2 g_loss : 1.318534
batch 3 d_loss : 0.378277
batch 3 g_loss : 1.523880

Epoch: 101
batch 1 d_loss : 0.355144
batch 1 g_loss : 1.186979
batch 2 d_loss : 0.488370
batch 2 g_loss : 1.720814
batch 3 d_loss : 0.357058
batch 3 g_loss : 1.242857

Epoch: 102
batch 1 d_loss : 0.386844
batch 1 g_loss : 1.488705
batch 2 d_loss : 0.499869
batch 2 g_loss : 1.346705
batch 3 d_loss : 0.401208
batch 3 g_loss : 1.243160

Epoch: 103
batch 1 d_loss : 0.345867
batch 1 g_loss : 1.154295
batch 2 d_loss : 0.393371
batch 2 g_loss : 1.049075
batch 3 d_loss : 0.525353
batch 3 g_loss : 1.814197

Epoch: 104
batch 1 d_loss : 0.362194
batch 1 g_loss : 1.589800
batch 2 d_loss : 0.402925
batch 2 g_loss : 1.220644
batch 3 d_loss : 0.445722
batch 3 g_loss : 1.434245

Epoch: 105
batch 1 d_loss : 0.443304
batch 1 g_loss : 1.275227
batch 2 d_loss : 0.501465
batch 2 g_loss : 1

batch 3 d_loss : 0.428766
batch 3 g_loss : 1.447939

Epoch: 149
batch 1 d_loss : 0.342884
batch 1 g_loss : 1.752414
batch 2 d_loss : 0.477634
batch 2 g_loss : 1.593410
batch 3 d_loss : 0.397671
batch 3 g_loss : 1.747224

Epoch: 150
batch 1 d_loss : 0.317331
batch 1 g_loss : 1.714910
batch 2 d_loss : 0.374779
batch 2 g_loss : 1.337041
batch 3 d_loss : 0.451159
batch 3 g_loss : 1.546246

Epoch: 151
batch 1 d_loss : 0.379576
batch 1 g_loss : 1.832867
batch 2 d_loss : 0.455636
batch 2 g_loss : 0.896191
batch 3 d_loss : 0.409353
batch 3 g_loss : 1.279328

Epoch: 152
batch 1 d_loss : 0.355198
batch 1 g_loss : 1.648393
batch 2 d_loss : 0.419709
batch 2 g_loss : 1.475408
batch 3 d_loss : 0.363895
batch 3 g_loss : 1.471596

Epoch: 153
batch 1 d_loss : 0.419628
batch 1 g_loss : 1.580718
batch 2 d_loss : 0.492294
batch 2 g_loss : 1.400119
batch 3 d_loss : 0.351757
batch 3 g_loss : 1.413004

Epoch: 154
batch 1 d_loss : 0.410887
batch 1 g_loss : 1.687140
batch 2 d_loss : 0.526860
batch 2 g_loss : 1

batch 3 d_loss : 0.311923
batch 3 g_loss : 1.654936

Epoch: 198
batch 1 d_loss : 0.364367
batch 1 g_loss : 2.039125
batch 2 d_loss : 0.381228
batch 2 g_loss : 1.232374
batch 3 d_loss : 0.499251
batch 3 g_loss : 2.690666

Epoch: 199
batch 1 d_loss : 0.421916
batch 1 g_loss : 1.723269
batch 2 d_loss : 0.361628
batch 2 g_loss : 1.399784
batch 3 d_loss : 0.362412
batch 3 g_loss : 1.280125

Epoch: 200
batch 1 d_loss : 0.276365
batch 1 g_loss : 1.336357
batch 2 d_loss : 0.390178
batch 2 g_loss : 1.586495
batch 3 d_loss : 0.344642
batch 3 g_loss : 1.803816

Epoch: 201
batch 1 d_loss : 0.343666
batch 1 g_loss : 1.558639
batch 2 d_loss : 0.342773
batch 2 g_loss : 1.276795
batch 3 d_loss : 0.317666
batch 3 g_loss : 1.548097

Epoch: 202
batch 1 d_loss : 0.363348
batch 1 g_loss : 1.905674
batch 2 d_loss : 0.455573
batch 2 g_loss : 1.714123
batch 3 d_loss : 0.331285
batch 3 g_loss : 1.718641

Epoch: 203
batch 1 d_loss : 0.273349
batch 1 g_loss : 1.427203
batch 2 d_loss : 0.345316
batch 2 g_loss : 1

batch 3 d_loss : 0.233471
batch 3 g_loss : 1.747631

Epoch: 247
batch 1 d_loss : 0.223537
batch 1 g_loss : 2.374846
batch 2 d_loss : 0.287277
batch 2 g_loss : 1.418445
batch 3 d_loss : 0.302320
batch 3 g_loss : 2.484096

Epoch: 248
batch 1 d_loss : 0.231061
batch 1 g_loss : 1.701074
batch 2 d_loss : 0.228210
batch 2 g_loss : 1.533550
batch 3 d_loss : 0.164732
batch 3 g_loss : 1.756343

Epoch: 249
batch 1 d_loss : 0.385257
batch 1 g_loss : 2.878498
batch 2 d_loss : 0.407627
batch 2 g_loss : 1.738209
batch 3 d_loss : 0.250462
batch 3 g_loss : 2.757827

Epoch: 250
batch 1 d_loss : 0.348315
batch 1 g_loss : 2.495444
batch 2 d_loss : 0.337432
batch 2 g_loss : 1.658700
batch 3 d_loss : 0.264050
batch 3 g_loss : 1.668651

Epoch: 251
batch 1 d_loss : 0.205338
batch 1 g_loss : 2.037641
batch 2 d_loss : 0.312640
batch 2 g_loss : 1.866976
batch 3 d_loss : 0.139804
batch 3 g_loss : 1.879774

Epoch: 252
batch 1 d_loss : 0.288898
batch 1 g_loss : 2.777081
batch 2 d_loss : 0.352752
batch 2 g_loss : 1

batch 3 d_loss : 0.142566
batch 3 g_loss : 2.838539

Epoch: 296
batch 1 d_loss : 0.253359
batch 1 g_loss : 3.060433
batch 2 d_loss : 0.288214
batch 2 g_loss : 2.213346
batch 3 d_loss : 0.161062
batch 3 g_loss : 2.368362

Epoch: 297
batch 1 d_loss : 0.139593
batch 1 g_loss : 2.129898
batch 2 d_loss : 0.135934
batch 2 g_loss : 1.593387
batch 3 d_loss : 0.279204
batch 3 g_loss : 3.547099

Epoch: 298
batch 1 d_loss : 0.192909
batch 1 g_loss : 2.476304
batch 2 d_loss : 0.166262
batch 2 g_loss : 1.735766
batch 3 d_loss : 0.240746
batch 3 g_loss : 2.995926

Epoch: 299
batch 1 d_loss : 0.132811
batch 1 g_loss : 1.755912
batch 2 d_loss : 0.169222
batch 2 g_loss : 2.638606
batch 3 d_loss : 0.121568
batch 3 g_loss : 2.562298

Epoch: 300
batch 1 d_loss : 0.113997
batch 1 g_loss : 2.262747
batch 2 d_loss : 0.207423
batch 2 g_loss : 2.591791
batch 3 d_loss : 0.142314
batch 3 g_loss : 2.185077

Epoch: 301
batch 1 d_loss : 0.196600
batch 1 g_loss : 2.519970
batch 2 d_loss : 0.197520
batch 2 g_loss : 2

batch 3 d_loss : 0.325196
batch 3 g_loss : 3.408610

Epoch: 345
batch 1 d_loss : 0.197151
batch 1 g_loss : 3.390933
batch 2 d_loss : 0.202729
batch 2 g_loss : 2.328582
batch 3 d_loss : 0.119360
batch 3 g_loss : 2.598208

Epoch: 346
batch 1 d_loss : 0.140669
batch 1 g_loss : 2.636992
batch 2 d_loss : 0.229004
batch 2 g_loss : 3.077944
batch 3 d_loss : 0.192690
batch 3 g_loss : 3.377566

Epoch: 347
batch 1 d_loss : 0.191905
batch 1 g_loss : 2.798266
batch 2 d_loss : 0.167548
batch 2 g_loss : 2.390188
batch 3 d_loss : 0.127387
batch 3 g_loss : 2.760846

Epoch: 348
batch 1 d_loss : 0.158932
batch 1 g_loss : 3.588716
batch 2 d_loss : 0.186348
batch 2 g_loss : 2.035718
batch 3 d_loss : 0.161131
batch 3 g_loss : 3.034944

Epoch: 349
batch 1 d_loss : 0.123107
batch 1 g_loss : 2.647936
batch 2 d_loss : 0.146449
batch 2 g_loss : 2.141827
batch 3 d_loss : 0.167361
batch 3 g_loss : 3.922840

Epoch: 350
batch 1 d_loss : 0.106131
batch 1 g_loss : 2.818577
batch 2 d_loss : 0.208403
batch 2 g_loss : 3

batch 3 d_loss : 0.131031
batch 3 g_loss : 3.004231

Epoch: 394
batch 1 d_loss : 0.107647
batch 1 g_loss : 3.046173
batch 2 d_loss : 0.163055
batch 2 g_loss : 2.573143
batch 3 d_loss : 0.145200
batch 3 g_loss : 3.108005

Epoch: 395
batch 1 d_loss : 0.164400
batch 1 g_loss : 2.961859
batch 2 d_loss : 0.160514
batch 2 g_loss : 2.288252
batch 3 d_loss : 0.165540
batch 3 g_loss : 2.950718

Epoch: 396
batch 1 d_loss : 0.083605
batch 1 g_loss : 2.566344
batch 2 d_loss : 0.144450
batch 2 g_loss : 2.466664
batch 3 d_loss : 0.108053
batch 3 g_loss : 3.438806

Epoch: 397
batch 1 d_loss : 0.195467
batch 1 g_loss : 4.552744
batch 2 d_loss : 0.198653
batch 2 g_loss : 2.301984
batch 3 d_loss : 0.071359
batch 3 g_loss : 2.772134

Epoch: 398
batch 1 d_loss : 0.206363
batch 1 g_loss : 3.187725
batch 2 d_loss : 0.179503
batch 2 g_loss : 2.177392
batch 3 d_loss : 0.099602
batch 3 g_loss : 3.135759

Epoch: 399
batch 1 d_loss : 0.099066
batch 1 g_loss : 2.364593
batch 2 d_loss : 0.196379
batch 2 g_loss : 3

## Generating Image

In [21]:
class ImageGenerator:
    
    def generate_image(self, batch_size):
        modelCreator = ModelCreator()
        
        generator = modelCreator.generator_model()
        generator.compile(loss=ENTROPY, optimizer=SGD_OPT)
        generator.load_weights(G_WGTS_FILE)
        
        imageProcessor = ImageProcessor()
        
        noise = np.zeros((batch_size, 100))
        Util().randomize_noise(noise, batch_size)
  
        generated_images = generator.predict(noise, verbose=1)
        
        print('generated_images.shape: {}'.format(generated_images.shape))
        
        for image in generated_images:
            image = image[0]
            image = image*127.5+127.5
            Image.fromarray(image.astype(np.uint8)).save("dirty.png")
            Image.fromarray(image.astype(np.uint8)).show()
            imageProcessor.clean(image)
            image = Image.fromarray(image.astype(np.uint8))
            image.show()        
            image.save("clean.png")

In [22]:
imageGenerator = ImageGenerator()

imageGenerator.generate_image(1)

1/1 [==============================] - 0s
generated_images.shape: (1, 1, 128, 128)
